<a href="https://colab.research.google.com/github/LeoD-h/Project-BCI-EEG/blob/main/TryMlModelLeo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# On désinstalle scikit-learn pour nettoyer le conflit
!pip uninstall scikit-learn -y

# On réinstalle la version qui marche bien avec MOABB
!pip install scikit-learn==1.5.2 mne moabb

Found existing installation: scikit-learn 1.5.2
Uninstalling scikit-learn-1.5.2:
  Successfully uninstalled scikit-learn-1.5.2
  Using cached scikit_learn-1.5.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.


In [2]:
import numpy as np

def generer_signal_truque(intention):
    """
    Génère un signal EEG aléatoire (bruit blanc), mais modifie
    l'amplitude des électrodes C3 et C4 pour simuler une intention.
    """
    fake_signal = np.random.normal(0, 1, size=(1, 22, 1000))

    # Indices des électrodes motrices (dans BCI IV 2a)
    # C3 = Index 7 (Cortex Moteur Gauche -> Gère Main Droite)
    # C4 = Index 11 (Cortex Moteur Droit -> Gère Main Gauche)

    if intention == "droite":
        # Pour bouger à DROITE, le Cortex Gauche (C3) travaille (Désynchronisation)
        # -> L'énergie baisse sur C3.
        # -> L'énergie reste normale/forte sur C4.

        fake_signal[0, 7, :] *= 0.1  # On éteint C3 (Amplitude très faible)
        fake_signal[0, 11, :] *= 2.0 # On allume C4 (Amplitude forte)
        print("Génération signal : Simulation 'Main Droite' (C3 faible, C4 fort)")

    elif intention == "gauche":
        # Inversement pour la GAUCHE
        fake_signal[0, 7, :] *= 2.0  # On allume C3
        fake_signal[0, 11, :] *= 0.1 # On éteint C4
        print("⚡ Génération signal : Simulation 'Main Gauche' (C3 fort, C4 faible)")

    return fake_signal

# --- 2. GÉNÉRATION ET TEST ---

print("--- TEST DE L'IA AVEC UN FAUX SIGNAL ---")

# A. On demande une MAIN DROITE
signal_droite = generer_signal_truque("droite")

prediction = mon_ia.predict(signal_droite)

print(f"\nL'IA analyse le signal...")
print(f"Résultat brut (Classe) : {prediction[0]}")

# Décodage (Rappel : 0=Gauche, 1=Droite)
if prediction[0] == 1:
    print("SUCCÈS ! L'IA a bien détecté la MAIN DROITE.")
else:
    print("ÉCHEC. L'IA s'est trompée (c'est normal, c'est du bruit synthétique).")

# --- 3. TEST INVERSE (Optionnel) ---
print("\n--- CONTRE-TEST (MAIN GAUCHE) ---")
signal_gauche = generer_signal_truque("gauche")
pred_gauche = mon_ia.predict(signal_gauche)

if pred_gauche[0] == 0:
    print("SUCCÈS ! L'IA a bien détecté la MAIN GAUCHE.")
else:
    print("Raté pour la gauche.")

--- TEST DE L'IA AVEC UN FAUX SIGNAL ---
⚡ Génération signal : Simulation 'Main Droite' (C3 faible, C4 fort)

🧠 L'IA analyse le signal...
👉 Résultat brut (Classe) : 1
✅ SUCCÈS ! L'IA a bien détecté la MAIN DROITE.

--- CONTRE-TEST (MAIN GAUCHE) ---
⚡ Génération signal : Simulation 'Main Gauche' (C3 fort, C4 faible)
✅ SUCCÈS ! L'IA a bien détecté la MAIN GAUCHE.
